In [ ]:
# 콜랩 사용 시 사용

from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 새 섹션

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, activations, initializers, losses, optimizers, metrics, regularizers


In [ ]:
# 압축 풀 경로 설정(colab)
# 음식이미지 폴더 생성 --> 압축 해제 후 저장 될 이미지 폴더
# 음식이미지 zip 폴더 생성 --> zip 파일 저장될 폴더

# zip 파일 // 음식 이미지 파일 구분해서 
# 명령어로 파일 이동이 되지 않는 관계로 압축 해제 후 수동으로 옮길것!!!(필수)
# 아래 주석으로 할 경우 체크포인트 발생으로 절대 쓰지 말 것!! 주석 풀린 것만 사용

!mkdir 음식이미지
!mkdir 음식이미지zip

!unzip -uq 'drive/MyDrive/음식이미지.zip' -d '음식이미지/'



# !mv 음식이미지_sample.zip 음식이미지/

# !cd 음식이미지 && unzip 음식이미지_sample.zip
# # !mv 음식이미지_sample.zip ./음식이미지zip/

# !cd ..

In [ ]:
import os
from os import listdir
from os.path import isfile, join
# tf.logging.set_verbosity(tf.logging.ERROR)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
# 음식이미지 내 폴더 명 확인 용도 (체크포인트 존재 유무 확인)

!ls '음식이미지/NewNewNewNew'

족발  보쌈	   파김치  불고기    연근조림  감자조림    콩나물무침
짬뽕  송편	   미역국  김치찜    꽁치조림  곱창전골    계란후라이
꿀떡  라면	   호박죽  꼬막찜    콩나물국  동그랑땡    오징어튀김
약과  육회	   물냉면  깍두기    장어구이  열무국수    애호박볶음
편육  물회	   산낙지  수제비    가지볶음  시래기국    오이소박이
수육  김밥	   감자탕  무생채    나박김치  계란말이    김치볶음밥
멍게  약식	   주먹밥  콩국수    제육볶음  고추튀김    코다리조림
쫄면  경단	   추어탕  회무침    새우튀김  유부초밥    순두부찌개
식혜  곰탕_설렁탕  생선전  계란국    닭볶음탕  배추김치    고등어조림
파전  떡국_만두국  칼국수  전복죽    갈치조림  양념치킨    고사리나물
잡채  육개장	   갈비찜  닭계장    양념게장  숙주나물    시금치나물
알밥  감자전	   막국수  비빔밥    잔치국수  땅콩조림    꽈리고추무침
무국  매운탕	   떡꼬치  계란찜    동태찌개  김치찌개    미역줄기볶음
순대  잡곡밥	   떡갈비  해물찜    총각김치  더덕구이    후라이드치킨
피자  콩자반	   갓김치  장조림    우엉조림  황태구이    메추리알장조림
한과  짜장면	   과메기  백김치    비빔냉면  멸치볶음
만두  김치전	   호박전  두부김치  두부조림  도토리묵
찜닭  수정과	   삼계탕  간장게장  부추김치  깻잎장아찌
젓갈  갈비탕	   북엇국  된장찌개  홍어무침  도라지무침


In [ ]:
# 메뉴명 리스트 생성

path = '음식이미지/NewNewNewNew'

menu = os.listdir(path) # 해당 폴더에 있는 파일 이름을 리스트 형태로 받음


len(menu)

129

In [ ]:
# 메뉴 파일 내 이미지 파일 이름 생성

menu_img_file_name = []

for i in menu:
    menu_img_file_name.append(os.listdir(path + '/' + i))

len(menu_img_file_name)

129

In [ ]:
## file 불러오기 test
## imread로 불러올 경우 경로 문제로 인하여 plt로 불러옴
## 모델의 구성 효과가 잘 나오지 않을 경우 RGB 채널 변환 시도해봐야
## data agmentation도 고려

menu_img = []

# for i in range(len(menu)):
#     for j in range(len(menu_img_file_name[i])):
#         file = path + '/' + menu[i] + '/' + menu_img_file_name[i][j]
#         f = cv2.imread(file)
#         menu_img.append(f)
#         print(f)

        
file = path + '/' + menu[1] + '/' + menu_img_file_name[1][0]
print(file)

# img = cv2.imread(file)
img = plt.imread(file)

img
# plt.figure(figsize=(10, 10))
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# plt.show()

음식이미지/NewNewNewNew/육개장/Img_020_0069.jpg


array([[[160,  85,   4],
        [160,  85,   2],
        [162,  88,   3],
        ...,
        [ 98,  55,  23],
        [ 93,  49,  20],
        [ 94,  50,  21]],

       [[160,  85,   2],
        [160,  85,   2],
        [161,  86,   3],
        ...,
        [ 96,  53,  21],
        [ 97,  54,  22],
        [ 94,  51,  19]],

       [[161,  86,   3],
        [161,  86,   3],
        [161,  86,   3],
        ...,
        [ 97,  54,  20],
        [102,  59,  27],
        [ 98,  55,  23]],

       ...,

       [[ 54,  59,  52],
        [ 56,  58,  53],
        [ 57,  59,  54],
        ...,
        [ 95,  39,  22],
        [ 95,  39,  22],
        [ 94,  38,  21]],

       [[ 58,  61,  54],
        [ 53,  55,  50],
        [ 52,  54,  49],
        ...,
        [ 94,  38,  21],
        [ 95,  39,  22],
        [ 94,  38,  23]],

       [[ 53,  55,  50],
        [ 52,  57,  51],
        [ 56,  61,  55],
        ...,
        [ 95,  39,  24],
        [ 95,  39,  24],
        [ 94,  38,  23]]

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale = 1./255)
 
# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        path,  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        class_mode='categorical')     # Categorical로

Found 11888 images belonging to 129 classes.


In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    
    # 
    tf.keras.layers.Dense(129, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [ ]:
# from tensorflow.keras.optimizers import RMSprop
 
# model.compile(loss='categorical_crossentropy',
#               optimizer=RMSprop(lr=0.001),
#               metrics=['acc'])

model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001),
              loss = keras.losses.CategoricalCrossentropy(),
              metrics = ['accuracy']) # tf.keras.metrics.CategoricalAccuracy()


In [ ]:
history = model.fit(
      train_generator,
      batch_size=50,
      steps_per_epoch=50,  
      epochs=25,
      verbose=1,
      )

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch 1/25
10/50 [=====>........................] - ETA: 7:02 - loss: 4.8619 - accuracy: 0.0063

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))


48/50 [===========================>..] - ETA: 20s - loss: 4.8065 - accuracy: 0.0131

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1078 bytes but only got 0. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 125 bytes but only got 120. Skipping tag 37510
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag)


50/50 [==============================] - 527s 10s/step - loss: 4.7950 - accuracy: 0.0133
Epoch 2/25
50/50 [==============================] - 525s 10s/step - loss: 4.4579 - accuracy: 0.0328
Epoch 3/25


/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 37386
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 37396
  " Skipping tag %s" % (size, len(data), tag)


50/50 [==============================] - 524s 10s/step - loss: 4.2400 - accuracy: 0.0650
Epoch 4/25
50/50 [==============================] - 527s 11s/step - loss: 3.9986 - accuracy: 0.0818
Epoch 5/25
50/50 [==============================] - 523s 10s/step - loss: 3.8018 - accuracy: 0.1126
Epoch 6/25
50/50 [==============================] - 526s 11s/step - loss: 3.5961 - accuracy: 0.1522
Epoch 7/25
50/50 [==============================] - 523s 10s/step - loss: 3.3938 - accuracy: 0.1823
Epoch 8/25
50/50 [==============================] - 524s 10s/step - loss: 3.2078 - accuracy: 0.2276
Epoch 9/25
50/50 [==============================] - 526s 10s/step - loss: 3.0010 - accuracy: 0.2624
Epoch 10/25
50/50 [==============================] - 527s 11s/step - loss: 2.7230 - accuracy: 0.3150
Epoch 11/25
50/50 [==============================] - 525s 10s/step - loss: 2.5161 - accuracy: 0.3658
Epoch 12/25
50/50 [==============================] - 525s 10s/step - loss: 2.2176 - accuracy: 0.4363
Epoch 13

In [ ]:
model.save('food_image_selector.h5')

In [ ]:
model.save('food_image_selector.json')

INFO:tensorflow:Assets written to: food_image_selector.json/assets


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
???

In [ ]:
???

In [ ]:
# 사진 입력시 출력 내용
# 파일명 및 경로 설정 필요

# file = path + '\\' + menu[1] + '\\' + menu_img_file_name[1][0]
file_name = 'Img_000_0000.jpg'

#path -> 입력 사진으로 변경
img = image.load_img(file, target_size=(300, 300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)

for i in range(len(classes[0])):
    if classes[0][i] == 1:
        print(file_name, '은/는 ', menu[i], ' 입니다.')

Img_000_0000.jpg 은/는  갈비탕  입니다.
